# Semantic LLM caching

## Colab-specific setup

Make sure you have a Database, a Secure Connect Bundle and a Database Token. See [DB Setup](https://cassio.org/db_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.
Get ready to upload the Bundle and supply the Token string.

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly. See [API Setup](https://cassio.org/api_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.

_Note: this colab is autogenerated from a [regular Jupyter notebook](https://cassio.org/frameworks/langchain/semantic-caching-llm-responses/) hosted on cassio.org. Run all cells in this section to complete the setup before moving on to the demo content proper._

_Note: you can work with your own Cassandra cluster instead of Astra DB, provided it is reachable from the cloud: check the commented code below for the case `cqlMode="local"` and consult [cassio.org](https://cassio.org/frameworks/langchain/setup/#database-choice) for more details._

In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio#egg=langchain" \
    "cassandra-driver>=3.28.0" \
    "cassio>=0.0.4" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_APPLICATION_TOKEN = input('Your Astra DB Token: ')

### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

### LLM Provider

In the cell below you can choose between **GCP VertexAI** or **OpenAI** for your LLM services.
Make sure you set the `llmProvider` variable and supply the corresponding access secrets.

In [ ]:
# Set your secret(s) for LLM access (key names must match `providerValidator` in `llm_choice`)
llmProvider = 'OpenAI'  # 'GCP_VertexAI'
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

ASTRA_DB_CLIENT_ID = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_APPLICATION_TOKEN,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    # elif mode == 'local':
    #     cluster = Cluster(
    #         ['192.168.0.1', '192.168.0.2'],
    #         auth_provider=PlainTextAuthProvider(
    #             "username",
    #             "password!",
    #         ),
    #     )
    #     # See https://docs.datastax.com/en/developer/python-driver/latest/getting_started/#connecting-to-cassandra for more options
    #     localSession = cluster.connect()
    #     return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    # elif mode == 'local':
    #     return <NAME_OF_YOUR_LOCAL_KEYSPACE>
    else:
        raise ValueError('Unknown CQL Session mode')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# Semantic LLM caching

_**NOTE:** this uses Cassandra's "Vector Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

The Cassandra-backed "semantic cache" for prompt responses is imported like this:

In [1]:
from langchain.cache import CassandraSemanticCache

As usual, a database connection is needed to access Cassandra. The following assumes
that a _vector-search-capable Astra DB instance_ is available. Adjust as needed.

In [2]:
# creation of the DB connection
cqlMode = 'astra_db' # alternatively, 'local' ... if you do have a Cassandra cluster to use, that is
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

An embedding function and an LLM are needed.

Below is the logic to instantiate the LLM and embeddings of choice. We choose to leave it in the notebooks for clarity.

In [3]:
# creation of the LLM resources


if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    myEmbedding = VertexAIEmbeddings()
    print('LLM+embeddings from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)
    myEmbedding = OpenAIEmbeddings()
    print('LLM+embeddings from OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from OpenAI


## Create the cache

At this point you can instantiate the semantic cache.

_Note: in the following it is made clear, through the `table_name_prefix` parameter, that different embeddings will require separate tables. This is done here to avoid mismatches when running this demo over and over with varying embedding functions: in most applications, where a single embedding suffices, there's no need to be this finicky._

In [5]:
cassSemanticCache = CassandraSemanticCache(
    session=session,
    keyspace=keyspace,
    embedding=myEmbedding,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

Make sure the cache starts empty with:

In [6]:
cassSemanticCache.clear_through_llm(llm=llm)

Configure the cache at a LangChain global level:

In [7]:
import langchain
langchain.llm_cache = cassSemanticCache

## Use the cache

Now try submitting a few prompts to the LLM and pay attention to the response times.

If the LLM is actually run, they should be the order of a few seconds; but in case of a cache hit, it will be way less than a second.

Notice that you get a cache hit even after rephrasing the question.

In [8]:
%%time
SPIDER_QUESTION_FORM_1 = "How many eyes do spiders have?"
# A new question should take long
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 12.3 ms, sys: 2.34 ms, total: 14.7 ms
Wall time: 2.18 s


'\n\nMost spiders have eight eyes.'

In [9]:
%%time
# Second time, very same question, this should be quick
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 5.22 ms, sys: 642 µs, total: 5.86 ms
Wall time: 263 ms


'\n\nMost spiders have eight eyes.'

In [10]:
%%time
SPIDER_QUESTION_FORM_2 = "How many eyes does a spider generally have?"
# Just a rephrasing: but it's the same question,
# so it will just take the time to evaluate embeddings
llm(SPIDER_QUESTION_FORM_2)

CPU times: user 5.95 ms, sys: 1.04 ms, total: 6.98 ms
Wall time: 436 ms


'\n\nMost spiders have eight eyes.'

Time for a really new question:

In [11]:
%%time
LOGIC_QUESTION_FORM_1 = "Is absence of proof the same as proof of absence?"
# A totally new question
llm(LOGIC_QUESTION_FORM_1)

CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 2.39 s


'\n\nNo, absence of proof is not the same as proof of absence.'

In [12]:
%%time
SPIDER_QUESTION_FORM_3 = "How many eyes are on the head of a typical spider?"
# Trying to catch the cache off-guard :)
llm(SPIDER_QUESTION_FORM_3)

CPU times: user 6.45 ms, sys: 1.29 ms, total: 7.74 ms
Wall time: 589 ms


'\n\nMost spiders have eight eyes.'

In [13]:
%%time
LOGIC_QUESTION_FORM_2 = "Is it true that the absence of a proof equates the proof of an absence?"
# Switching to the other question again
llm(LOGIC_QUESTION_FORM_2)

CPU times: user 12.5 ms, sys: 1.17 ms, total: 13.7 ms
Wall time: 537 ms


'\n\nNo, absence of proof is not the same as proof of absence.'

## Additional options

When creating the semantic cache, you can specify a few other options such as the metric used to calculate the similarity and the number of entries to retrieve in the ANN step (i.e. those on which the exact requested metric is computed for the final filtering). Here is an example which uses the L2 metric:

In [14]:
anotherCassSemanticCache = CassandraSemanticCache(
    session=session,
    keyspace=keyspace,
    embedding=myEmbedding,
    distance_metric='l2',
    score_threshold=0.4,
    num_rows_to_fetch=12,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

This cache builds on the same database table as the previous one, as can be seen e.g. with:

In [15]:
lookup = anotherCassSemanticCache.lookup_with_id_through_llm(
    LOGIC_QUESTION_FORM_2,
    llm,
)
if lookup:
    docId, response = lookup
    print(docId)
    print(response)
else:
    print('No match.')

77add13036bcaa23c74ebf2ab2c56441
[Generation(text='\n\nNo, absence of proof is not the same as proof of absence.', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n\nNo, absence of proof is not the same as proof of absence.', generation_info={'finish_reason': 'stop', 'logprobs': None})]


## Stale entry control

### Time-To-Live (TTL)

You can configure a time-to-live property of the cache, with the effect of automatic eviction of cached entries after a certain time.

Setting `langchain.llm_cache` to the following will have the effect that entries vanish in an hour:

In [16]:
cacheWithTTL = CassandraSemanticCache(
    session=session,
    keyspace=keyspace,
    embedding=myEmbedding,
    ttl_seconds=15,
    table_name_prefix=f'semantic_cache_{llmProvider}_',
)

### Manual cache eviction

Alternatively, you can invalidate individual entries one at a time, just like you saw for the exact-match `CassandraCache` cache.

But this is an index based on sentence similarity, so this time the procedure has two steps: **first**, a lookup to find the _id_ of the matching document:

In [17]:
lookup = cassSemanticCache.lookup_with_id_through_llm(SPIDER_QUESTION_FORM_1, llm)
if lookup:
    docId, response = lookup
    print(docId)
else:
    print('No match.')

0a1339bc659790da078a4352c05bf422


_you can see that querying for another form for the "same" question will result in the same id:_

In [18]:
lookup2 = cassSemanticCache.lookup_with_id_through_llm(SPIDER_QUESTION_FORM_2, llm)
if lookup:
    docId2, response2 = lookup2
    print(docId2)
else:
    print('No match.')

0a1339bc659790da078a4352c05bf422


and **second**, the document id is used in the actual cache eviction (again, you have to additionally provide the LLM):

In [19]:
cassSemanticCache.delete_by_document_id_through_llm(docId, llm)

As a check, try asking that question again:

In [20]:
%%time
llm(SPIDER_QUESTION_FORM_1)

CPU times: user 22.7 ms, sys: 1.21 ms, total: 23.9 ms
Wall time: 1.27 s


'\n\nSpiders generally have eight eyes, but some have fewer.'

### Whole-cache deletion

Lastly, as you have seen earlier, you can empty the cache entirely, for a given LLM, with:

In [21]:
cassSemanticCache.clear_through_llm(llm=llm)